In [1]:
import os
import cv2
import json
import numpy as np
import shutil
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


/Users/vineetkumar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
print("TrashNet exists:", os.path.isdir("dataset-resized"))
print("TACO exists:", os.path.isdir("taco_classes"))


TrashNet exists: True
TACO exists: True


In [3]:
os.makedirs("merged_dataset", exist_ok=True)
print("Created: merged_dataset")


Created: merged_dataset


In [4]:
def merge_folder(src, dst):
    os.makedirs(dst, exist_ok=True)
    for file in os.listdir(src):
        src_file = os.path.join(src, file)
        dst_file = os.path.join(dst, file)
        if os.path.isfile(src_file):
            shutil.copy(src_file, dst_file)

trashnet_root = "dataset-resized"

for cls in os.listdir(trashnet_root):
    src = os.path.join(trashnet_root, cls)
    dst = os.path.join("merged_dataset", cls)
    if os.path.isdir(src):
        merge_folder(src, dst)

print("TrashNet merged successfully!")


TrashNet merged successfully!


In [5]:
taco_root = "taco_classes"

for cls in os.listdir(taco_root):
    src = os.path.join(taco_root, cls)
    dst = os.path.join("merged_dataset", cls)
    if os.path.isdir(src):
        merge_folder(src, dst)

print("TACO merged successfully!")


TACO merged successfully!


In [6]:
IMG_SIZE = 224
X, y = [], []
label_map = {}
label_id = 0

for cls in sorted(os.listdir("merged_dataset")):
    cls_path = os.path.join("merged_dataset", cls)
    if not os.path.isdir(cls_path):
        continue

    label_map[label_id] = cls

    for img_file in os.listdir(cls_path):
        img_path = os.path.join(cls_path, img_file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(label_id)

    label_id += 1

X = np.array(X) / 255.0
y = np.array(y)

print("Total images:", len(X))
print("Classes found:", label_map)


Total images: 7126
Classes found: {0: 'Aerosol', 1: 'Aluminium blister pack', 2: 'Aluminium foil', 3: 'Battery', 4: 'Broken glass', 5: 'Carded blister pack', 6: 'Cigarette', 7: 'Clear plastic bottle', 8: 'Corrugated carton', 9: 'Crisp packet', 10: 'Disposable food container', 11: 'Disposable plastic cup', 12: 'Drink can', 13: 'Drink carton', 14: 'Egg carton', 15: 'Foam cup', 16: 'Foam food container', 17: 'Food Can', 18: 'Food waste', 19: 'Garbage bag', 20: 'Glass bottle', 21: 'Glass cup', 22: 'Glass jar', 23: 'Magazine paper', 24: 'Meal carton', 25: 'Metal bottle cap', 26: 'Metal lid', 27: 'Normal paper', 28: 'Other carton', 29: 'Other plastic', 30: 'Other plastic bottle', 31: 'Other plastic container', 32: 'Other plastic cup', 33: 'Other plastic wrapper', 34: 'Paper bag', 35: 'Paper cup', 36: 'Paper straw', 37: 'Pizza box', 38: 'Plastic bottle cap', 39: 'Plastic film', 40: 'Plastic glooves', 41: 'Plastic lid', 42: 'Plastic straw', 43: 'Plastic utensils', 44: 'Plastified paper bag', 4

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [8]:
base = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))
base.trainable = False

x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
output = Dense(len(label_map), activation="softmax")(x)

model = Model(inputs=base.input, outputs=output)

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,602,882 (9.93 MB)

 Trainable params: 344,898 (1.32 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [9]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 118s 632ms/step - accuracy: 0.3132 - loss: 2.7640 - val_accuracy: 0.3850 - val_loss: 2.3018
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 95s 528ms/step - accuracy: 0.4508 - loss: 2.0155 - val_accuracy: 0.3899 - val_loss: 2.2815
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 87s 485ms/step - accuracy: 0.5078 - loss: 1.7283 - val_accuracy: 0.4165 - val_loss: 2.3322
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 103s 575ms/step - accuracy: 0.5722 - loss: 1.5075 - val_accuracy: 0.4144 - val_loss: 2.3864
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 93s 517ms/step - accuracy: 0.6359 - loss: 1.2717 - val_accuracy: 0.4074 - val_loss: 2.4447
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 99s 551ms/step - accuracy: 0.6820 - loss: 1.1021 - val_accuracy: 0.4046 - val_loss: 2.5556
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 94s 524ms/step - accuracy: 0.7402 - loss: 0.9148 - val_accuracy: 0.3871 - val_loss: 2.6945
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 105s 590ms/step - accuracy: 0.7943 - loss

In [10]:
model.save("waste_model_merged.keras")

with open("label_map_merged.json", "w") as f:
    json.dump(label_map, f)

print("Model + label map saved!")


Model + label map saved!
